# NEPOOL IDR Drop

Implements class to batch tasks used for IDR drops.

In [1]:
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
import selenium.webdriver.support.ui as ui
from selenium.webdriver.common.keys import Keys
import selenium.webdriver as webdriver
from pandas.io.json import json_normalize
import datetime as dt
import pandas as pd
import numpy as np
import IDRdrop
import EPOwebscrape
#import emailscrape
import json
import os
basepath = '/home/jupyter-engiela/la-tools-test/IDR_Drop'
print('working in directory {}.'.format(basepath))

working in directory /home/jupyter-engiela/la-tools-test/IDR_Drop.


In [2]:
def bodies_json(bodies):

    test = pd.DataFrame.from_dict(bodies, orient = 'index')

    if type(test.date[0]) == str:
            test.date = pd.to_datetime(test.date)

    last_days = max(test.date) - dt.timedelta(7) 

    sub = test[test.date > last_days]
        
    accts_success = [len(accts) > 0 for accts in sub.accts]
    accts_fail = [not val for val in accts_success]
        
    good = sub[accts_success].reset_index(drop = True)

    email_error = []

    if len(accts_fail) > 0:
        bad = sub[accts_fail].reset_index()
        mail_error = 'EMAIL_SCRAPE_ERROR.csv'

        bad.to_csv(mail_error, header = True, index = False)

    return(good)


#output_dict, filename = emailscrape.get_emails()
with open('email_bodies_05_30_2019.json', 'r') as f:
    bodies = json.load(f)

    bodies = json.loads(bodies)


good = bodies_json(bodies)

print(good.head())
print('...')
print(good.tail())

                               accts                date  \
0             [800030802, 800194201] 2019-05-30 11:44:39   
1             [800537201, 800581301] 2019-05-30 11:42:43   
2                        [800166101] 2019-05-30 11:38:45   
3                        [800582101] 2019-05-29 09:04:42   
4  [800497301, 800497304, 800497305] 2019-05-29 09:00:45   

                      name      pw           user  
0              NASHUA YMCA  7m5f7f  nhengi-nash3n  
1     PORTWALK HAMPTON INN  2n5a2a  nhengi-hamp8n  
2              OATS REALTY  zu43z8  nhengi-oats6n  
3  BRITTON LUMBER COMPANY,  4s4e7s  nhengi-brit8n  
4          TEXTILES COATED  7u4n4f  nhengi-text9n  
...
                       accts                date  \
59              [5334750001] 2019-05-28 14:49:15   
60  [0167372027, 0167377022] 2019-05-24 12:53:35   
61              [9102996055] 2019-05-24 15:47:51   
62              [2876745007] 2019-05-24 15:49:56   
63              [7554298017] 2019-05-24 16:48:08   

          

## Download files from EPO portal

In [3]:
for row in range(0, len(good.accts)):
    #try:
    EPOwebscrape.idr_download(row, good)
    #except:
        #print('error, accts {}.'.format(good.accts))

WebDriverException: Message: unknown error: Chrome failed to start: exited abnormally
  (unknown error: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /usr/bin/google-chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)


## Show downloaded files from EPO portal

Here *filepath* is a directory containing downloaded EPO files. Code will print 20 most recent files.

In [4]:
#filepath = os.getcwd()
readpath = '/Users/stevenhurwitt/Downloads'

myfiles = IDRdrop.show_dir(readpath, 20)
print(myfiles)

files found in dir:  /Users/stevenhurwitt/Downloads
                                                files                time
0   ngrid_b3225013-4439-45ae-a205-95983fe51c20_dae... 2019-09-01 21:59:33
1   ngrid_389a101a-0dcf-4027-908e-927561fd3f3a_ca3... 2019-09-01 21:57:39
2   ngrid_18f051fb-1aad-40a3-ae59-2abc659323a1_bef... 2019-09-01 21:55:44
3   ngrid_b65a2843-1e0d-4007-bcec-240bb80844ac_825... 2019-09-01 21:53:20
4   ngrid_4dfb5eb6-0878-4107-aee0-b54a53523487_bcc... 2019-09-01 16:05:15
5   ngrid_23358185-c1fc-48fb-b878-ff809814525a_484... 2019-09-01 15:37:46
6                                                .git 2019-08-30 19:30:30
7                     ENGIE - Ewx Debugging Guide.pdf 2019-08-28 09:36:13
8               Energyworx Datalab API training.ipynb 2019-08-28 09:28:09
9                              Westchester Ave 90.m4a 2019-08-26 20:49:52
10                                          .DS_Store 2019-08-26 20:00:12
11                                  cr2_Blue_Form.pdf 2019-0

Choose files to split into Raw IDR files.

In [5]:
index = [0]

splitfiles = list(myfiles.files[:6])
print('files to split: ')
print(splitfiles)

files to split: 
['ngrid_b3225013-4439-45ae-a205-95983fe51c20_dae9a93c_hourlycsv.csv', 'ngrid_389a101a-0dcf-4027-908e-927561fd3f3a_ca3abaec_hourlycsv.csv', 'ngrid_18f051fb-1aad-40a3-ae59-2abc659323a1_bef34c8a_hourlycsv.csv', 'ngrid_b65a2843-1e0d-4007-bcec-240bb80844ac_8252fe57_hourlycsv.csv', 'ngrid_4dfb5eb6-0878-4107-aee0-b54a53523487_bcc3a42c_hourlycsv.csv', 'ngrid_23358185-c1fc-48fb-b878-ff809814525a_4842804b_hourlycsv.csv']


## Batch process downloaded EPO files into Raw IDRs

In [6]:
readpath = '/Users/stevenhurwitt/Downloads'
writepath = '/Users/stevenhurwitt/LA-tools-master/IDR_Drop/Raw IDR Data'
utility = 'MECO'
error_log = []

for file in splitfiles:
    #try:
    os.chdir(readpath)
    filedf = pd.read_csv(file, sep = ",", header = 0)
    
    IDRdrop.raw_split(filedf, readpath, writepath, utility)
    print('success, file: ', file)
        
    #except:
        #print('error, file: ', file)

found 4 accounts.
0347780009_MECO_IDR_RAW.csv  written.
3940856005_MECO_IDR_RAW.csv  written.
7678649009_MECO_IDR_RAW.csv  written.
9052467007_MECO_IDR_RAW.csv  written.
success, file:  ngrid_b3225013-4439-45ae-a205-95983fe51c20_dae9a93c_hourlycsv.csv
found 1 accounts.
0169384016_MECO_IDR_RAW.csv  written.
success, file:  ngrid_389a101a-0dcf-4027-908e-927561fd3f3a_ca3abaec_hourlycsv.csv
found 5 accounts.
success, file:  ngrid_18f051fb-1aad-40a3-ae59-2abc659323a1_bef34c8a_hourlycsv.csv
found 1 accounts.
5373369008_MECO_IDR_RAW.csv  written.
success, file:  ngrid_b65a2843-1e0d-4007-bcec-240bb80844ac_8252fe57_hourlycsv.csv
found 4 accounts.
0021483009_MECO_IDR_RAW.csv  written.
0040677012_MECO_IDR_RAW.csv  written.
success, file:  ngrid_4dfb5eb6-0878-4107-aee0-b54a53523487_bcc3a42c_hourlycsv.csv
found 4 accounts.
success, file:  ngrid_23358185-c1fc-48fb-b878-ff809814525a_4842804b_hourlycsv.csv


## Show Raw IDR files based on utility

In [3]:
utility = "CLP"

Here *rawpath* is directory containing Raw IDRs - 25 most recent will be shown.

In [7]:
rawfiles = IDRdrop.show_dir(writepath, 25)
print(rawfiles)

files found in dir:  /Users/stevenhurwitt/LA-tools-master/IDR_Drop/Raw IDR Data
                                             files                time
0                      0040677012_MECO_IDR_RAW.csv 2019-09-01 22:15:41
1                      0169384016_MECO_IDR_RAW.csv 2019-09-01 22:15:41
2                      0021483009_MECO_IDR_RAW.csv 2019-09-01 22:15:41
3                      9052467007_MECO_IDR_RAW.csv 2019-09-01 22:15:41
4                      3940856005_MECO_IDR_RAW.csv 2019-09-01 22:15:41
5                      0347780009_MECO_IDR_RAW.csv 2019-09-01 22:15:41
6                      5373369008_MECO_IDR_RAW.csv 2019-09-01 22:15:41
7                      7678649009_MECO_IDR_RAW.csv 2019-09-01 22:15:41
8                       0021483009MECO_IDR_RAW.csv 2019-09-01 22:14:51
9                       3940856005MECO_IDR_RAW.csv 2019-09-01 22:14:51
10                      7678649009MECO_IDR_RAW.csv 2019-09-01 22:14:51
11                      0040677012MECO_IDR_RAW.csv 2019-09-01 22:14:

Choose Raw IDRs to filter into IDR files.

In [9]:
index = [36]

processfiles = list(rawfiles.files[:8])
print('files to processed: ')
print(processfiles)

files to processed: 
['0040677012_MECO_IDR_RAW.csv', '0169384016_MECO_IDR_RAW.csv', '0021483009_MECO_IDR_RAW.csv', '9052467007_MECO_IDR_RAW.csv', '3940856005_MECO_IDR_RAW.csv', '0347780009_MECO_IDR_RAW.csv', '5373369008_MECO_IDR_RAW.csv', '7678649009_MECO_IDR_RAW.csv']


## Batch filter Raw IDR into IDR files to be dropped

In [17]:
readpath = '/Users/stevenhurwitt/LA-tools-master/IDR_Drop/Raw IDR Data'
writepath = '/Users/stevenhurwitt/LA-tools-master/IDR_Drop/IDR Data/NEPOOL/'
error_log = []

for dropfile in processfiles:
    try:
        IDRdrop.data_drop(dropfile, readpath, writepath)
        print('success, file: ', dropfile)
    
    except:
        error_log = error_log.append(dropfile)
        print("error, file: ", dropfile)

success, file:  0040677012_MECO_IDR_RAW.csv
success, file:  0169384016_MECO_IDR_RAW.csv
success, file:  0021483009_MECO_IDR_RAW.csv
success, file:  9052467007_MECO_IDR_RAW.csv
success, file:  3940856005_MECO_IDR_RAW.csv
success, file:  0347780009_MECO_IDR_RAW.csv
success, file:  5373369008_MECO_IDR_RAW.csv
success, file:  7678649009_MECO_IDR_RAW.csv


In [ ]:
error_log

In [18]:
dir(EPOwebscrape)

['BeautifulSoup',
 'Chrome',
 'Keys',
 'Options',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'acct_match',
 'big_match',
 'check_the_box',
 'export_data',
 'json',
 'json_normalize',
 'logon',
 'np',
 'os',
 'pd',
 'ui',
 'webdriver']